# ALI numerical algorithm for two-stream transfer

## Imports

In [ ]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

## 1) Set parameters, variables, and boundary values

### Set the grid

In [ ]:
# Set the number of grid points
n = 70
# For a few of the applications we need N+1 things
N = n+1

# Set the 1D grid
z = np.linspace(0.,1.,num=N)

# Set delta z (grid spacing value)
del_z = z[1]

### Gas variables

In [ ]:
# Set the thermal source function (aka the Planck function)
# In this example it is constant at each depth
B = np.ones(n)

# Photon destruction probability
# In this example it is constant 
epsilon = np.ones(n)*0.001

### Optical depth

In [ ]:
# Change in optical depth (delta tau)
# Right now it's set based on example 4.4.5
# but there will be more options to 
# calculate this depending on what 
# information is given in the problem
# It is also constant here because our
# grid spacing is equal

# The extinction coefficient is
# 10.^(5. - 6.z)

# Change this wording
def tau_fn(z_arr):
    """
    Input:
        z_arr = grid
    Output:
        tau = array of the optical depth at each grid point
    """
    # Get the length, delta z, and set the array for delta tau
    # All based on the array given
    l = len(z)
    delta_z = abs(z_arr[1]-z_arr[0])
    tau_arr = np.zeros((l))
    for i in range(0,l):
        j = z_arr[i] + delta_z/2.
        tau_arr[i] = np.sqrt(3.)*(10.**(5. - 6.*j))*delta_z
    return(tau_arr)

In [ ]:
tau = tau_fn(z)

## 2) Create interpolation functions and a matrix solver

### Quadratic Bezier Interpolation for I(U)

In [ ]:
# Interpolation coefficients 
# Only need interpolation if working in 2D or 3D
# Still not exactly sure how to define t
#t = 0.5
#m = 1. - t

#def I_interp(i,j):
#    """
#    Input: location on grid
#        i = x-coordinate 
#        j = y-coordinate
#    Output: Value of thrid-order-quadrature specific intensity value 
#        at the input location on grid
#    """
#    return(I[i-1,j]*u**2 + I[i+1,j]*2.*u*t + I[i+2,j]*t**2)

### Third order quadratic interpolation for the source contribution

In [ ]:
# Interpolation coefficients
e0 = np.ones((N)) - np.exp(-tau)
e1 = tau - e0
e2 = tau**2 - (2.*e1)

# Initialize the arrays for the coefficients
u_p = np.zeros((n))
p_p = np.zeros((n))
d_p = np.zeros((n))
u_m = np.zeros((n))
p_m = np.zeros((n))
d_m = np.zeros((n))

# Equations 3.40, 3.41, 3.42 from http://ita.uni-heidelberg.de/~dullemond/lectures/radtrans_2012/Chapter_3.pdf
for i in range(0,n):
    # Handle overshoot and undershoot of interpolation
    
    # I_+ coefficients, n of them
    u_p[i] = e0[i] + (e2[i] - (2.*tau[i] + tau[i+1])*e1[i])/(tau[i]*(tau[i]+tau[i+1]))
    p_p[i] = (((tau[i]+tau[i+1])*e1[i])-e2[i])/(tau[i]+tau[i+1])
    d_p[i] = (e2[i] - tau[i]*e1[i])/(tau[i+1]*(tau[i]+tau[i+1]))
    # I_- coefficients, n of them
    # Actually derive the equation for these...
    u_m[i] = e0[i] + (e2[i] - (2.*tau[i] + tau[i-1])*e1[i])/(tau[i]*(tau[i]+tau[i-1]))
    p_m[i] = (((tau[i]+tau[i-1])*e1[i])-e2[i])/(tau[i]+tau[i-1])
    d_m[i] = (e2[i] - tau[i]*e1[i])/(tau[i-1]*(tau[i]+tau[i-1]))

# Define the 1D source term interpolation function
def S_interp(i,S,plus=True):
    """
    Input: 
        i = location on grid, make sure to not exceed i=n-2
        S = Source value at each gridpoint
        plus = True if we are dealing with the ray that
               is being integrated from the bottom to top
               False if integrated from the top to bottom
    Output: Value of thrid-order-quadrature source value
            for the two-stream approximation
        at the input location on grid
    """
    
    if plus==True:
        # This calculates u_(+,i+1/2), p_(+,i+1/2), d_(+,i+1/2)
        return(u_p[i]*S[i-1] + p_p[i]*S[i] + d_p[i]*S[i+1])
    elif plus==False:
        # This calculates u_(-,i+1/2), p_(-,i+1/2), d_(-,i+1/2)
        return(u_m[i]*S[i+1] + p_m[i]*S[i] + d_m[i]*S[i-1])
    # Check to make sure that for I_- that it doesn't
    # go negative. See discussion around 3.46

### Partial Lambda Operator

In [ ]:
# These are the lower diagonal,
# diagonal, and upper diagonal
# pieces of the simplified lambda
# operator. These will vary across 
# the grid if tau varies with z

# Write as lambda* because Brant doesn't like just lambda

v = len(u_p)

# Lower Diagonal 
lmbda_l = 0.5*(u_p+d_m)

# Diagonal 
lmbda_d = 0.5*(p_p+p_m)

# Upper Diagonal
lmbda_u = 0.5*(d_p+u_m)

# Construct the actual matrix
# Needed for updating the source function later
lmbda_star = np.zeros((n,n))
for i in range(0,n):
    if i==0:
        lmbda_star[i,i]=lmbda_d[i]
        lmbda_star[i,i+1]=lmbda_u[i]
    elif i==(n-1):
        lmbda_star[i,i]=lmbda_d[i]
        lmbda_star[i,i-1]=lmbda_l[i]
    else:
        lmbda_star[i,i-1]=lmbda_l[i]
        lmbda_star[i,i]=lmbda_d[i]
        lmbda_star[i,i+1]=lmbda_u[i]

## M matrix

In [ ]:
# M* = [Identity - (1-epsilon)*Lambda*]
# Where Lambda* is the partial lambda operator 
# The matrix equation that will be solved is:
# M*S = epsilon*B
#M = np.identity(n) - lmbda*(np.ones((n))-epsilon)


# Lower Diagonal of M
M_a =  -(np.ones((v))-epsilon[0:v])*lmbda_l

# Diagonal 
M_b = np.ones((v)) - (np.ones((v))-epsilon[0:v])*lmbda_d

# Upper Diagonal 
M_c =  -(np.ones((v))-epsilon[0:v])*lmbda_u

## The tri-diagonal solver

In [ ]:
n# Thomas Algorithm if diagonally dominant
# Forward elimination for a tridiagonal matrix
# Based on the algorithm found in Dullemond Ch4 notes
# Backward Substition is then applied
def tri_solver(a,b,c,y,n):
    """
    Input: 
        a = upper diagonals of matrix M
        b = diagonals of matrix M
        c = lower diagonals of matrix M
        y = the solution array in the matrix equation: Mx = y (nx1)
        n = the number of rows in matrix m
    Output:
        A new matrix or vector of S values 
    """
    
    # Check to see if matrix is diagonally dominant
    flag = 0
    comp_arr = a[:-1] + c[1:] #Add the upper and lower diagonals
    dim = len(comp_arr)
    for i in range(0,dim):
        if comp_arr[i]>=c[i+1]: #See if the sum of the upper and lower diagonals are larger than the diagonal element
            flag = 1 # Not diagonally dominant
            break # If there is even one row that isn't diagonally dominant, we can't use the Thomas Algorithm
            
    # The Thomas Algorithm for diagonally dominant matrices
    if flag == 0:
        #-----------------------------------------------------------
        # Perform forward elimination and backward substitution on M
        # Thomas Algorithm: a simplified form of Gaussian Elimination
        # specifically for tridiagonal matrices 
        
        # Define column arrays for necessary row-specific constants
        # This automatically sets gamma[0] = beta[0] = 0 which is 
        # necessary for the problem
        g = np.zeros((n))
        beta = np.zeros((n))
    
        for i in range(0,n-1):
            g[i+1] = (-c[i]/(a[i]*g[i] + b[i]))
            beta[i+1] = (y[i] - beta[i]*a[i])/b[i]
    
        # Use the gamma and beta constant arrays to compute 
        # what is essentially a back substitution to finish the
        # calculation of MX = Y
        X = np.zeros((n+1))
        #X[0:n] = S
        for k in range(1,n+1):
            # Actually need to go from n, n-1, n-2,...,1
            # So create a new place holder that goes backwards
            c = n - k #Count backwards
            X[c] = g[c]*X[c+1] + beta[c]
    
        # Return the new solution to J
        return(X[0:n])
    
    # Non-diagonally dominant matrices
    else:
        for i in range(1,n-1):
            w = c[i]/b[i-1]
            b[i] = b[i] - w*a[i-1]
            y[i] = y[i] - w*y[i-1]
        X = np.zeros((n))
        X[n-1] = y[n-1]/b[n-1]
        for k in range(2,n):
            m = n - k 
            X[m] = (y[m] - a[m]*X[m+1])/b[m]
        return(X)

## 3) Solve for converged S

In [ ]:
# Initialize I+ and I-
# These are not initial values
I_plus = np.zeros((n-1))
I_plus[0] = 1.
I_minus = np.zeros((n-1))

# Start with an initial guess for J
# The mean specific intensity
J = np.zeros((n))
J[0] = 1.

# Set the initial source function
S = epsilon*B + (np.ones(n) - epsilon)*J

# Calculate Y = epsilon*B + (1 - epsilon)(J - (lambda*)*S)
# Need to carefully use np.matmul to make sure that everything 
# gets multiplied through correctly
Y = epsilon*B + (np.ones(n) - epsilon)*(J - np.transpose(np.matmul(lmbda_star,S)))

# Calculate emissivity across the grid
# 3.10 Chap.3 radtrans notes, website in the above section: Third order quadratic integration...
# j = B*alpha 
# alpha = 10.**(5. - 6.*z)
j = B*(10.**(5. - 6.*z[0:-1]))

# Calculate the maximum quadrature limiter
# This prevents non-physical occurrences 
# during the interpolation
# Eqn 3.37 of chap.3 of radtrans notes
#Q_max = 

for j in range(0,101):
    
    # Solve for I
    # I = exp(-tau)*I_prev + S_interp
    # ----------------------------------------------------------------------
    # For 2D and 3D use the function defined for 
    # quadratic Bezier interpolation to find the new value for I(U)
    # For 1D just use the i-1 grid point for the exponential term
    # Use a different third-order-quadrature method for S contributions
    for i in range(1,n-1):
        I_plus[i] = np.exp(-tau[i-1])*I_plus[i-1] + S_interp(i,S,plus=True)
        I_minus[i] = np.exp(-tau[i])*I_minus[i] + S_interp(i,S,plus=False)
        
        
    # Solve for J
    # J = 0.5*int_(-1)^(1) I(mu)*d mu
    # ----------------------------------------------------------------------
    J[1:] = 0.5*(I_plus + I_minus)
    
    # Solve for S
    # ----------------------------------------------------------------------
    # The 1D diffusion equation is a 2nd order PDE where lambda[S] is the solution
    # The numerical representation used for this PDE is Central Space
    # Forward substitution + backward substitution are used to solve the PDE
    S = epsilon*B + (np.ones(n) - epsilon)*J
    S = tri_solver(M_a,M_b,M_c,Y,n)
        
print(j)

In [ ]:
y = S/B
z = z[:-1]
plt.semilogy(z,y)
plt.title("S/B vs z, epsilon=%1.3f" %epsilon[0])
plt.xlabel("z")
plt.ylabel("S/B")
#plt.ylim(0.001,1.2)
plt.xlim(-0.1,1.0)
plt.show()